# Funciones

In [3]:
import cv2 as cv
import numpy as np
import imutils
import SimpleITK as sitk
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, RadioButtons, Checkbox

## Código para subir imagen

In [ ]:
imagen = cv.imread('/imagenes/imagen.jpg', cv.IMREAD_GRAYSCALE)
imagen_grises = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
_,imagen_bin = cv.threshold(imagen_grises,150,255,cv.THRESH_BINARY_INV) # tambien THRESH_BINARY sin INV

## Aplicar máscara

In [ ]:
mask = np.zeros(imagen.shape, dtype=np.uint8)   # dtype: 'int8', bool, float, tener en cuenta el tipo de dato al realizar las operaciones
imagen_masked = cv.bitwise_and(imagen,mask)     # and entre imagen y máscara
imagen_masked2 = cv.bitwise_and(imagen,imagen,mask=mask)         # se hace el AND sólo en los px definidos por la máscara

## Umbralizado

In [ ]:
def umbralBinario(img,umbral):
    _, img_umbralizada = cv.threshold(img, umbral, 255, cv.THRESH_BINARY)
    return img_umbralizada

def umbralAdaptativo(img,maximo,tamanio_bloque,c):
    return cv.adaptiveThreshold(img,maximo, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, tamanio_bloque, c);

## Perfil de intensidad

In [ ]:
# PERFIL DE INTENSIDAD PARA PUNTO DE UNA IMAGEN
x=200
y=200
i = imagen[x,y]

print(f"Intensidad en el punto ({x},{y}): {i}")

In [ ]:
# PERFIL DE INTENSIDAD PARA FILA DE UNA IMAGEN
fila=200
fig,ax = plt.subplots(1,2)
fig.set_figheight(4)
fig.set_figwidth(16)
ax[0].imshow(imagen,cmap='gray')
ax[0].plot([0,imagen.shape[1]],[fila,fila])
ax[1].plot(imagen[fila,:])
plt.tight_layout()
plt.show()

In [ ]:
# PERFIL DE INTENSIDAD PARA COLUMNA DE UNA IMAGEN
columna=200
fig,ax = plt.subplots(1,2, figsize=(16,4))
ax[0].imshow(imagen,cmap='gray')
ax[0].plot([columna,columna],[imagen.shape[0],0])
ax[1].plot(imagen[:,columna])
plt.tight_layout()
plt.show()

In [ ]:
# GRAFICO PERFIL DE INTENSIDAD PARA ROI (REGION DE INTERÉS) DE UNA IMAGEN
x = 100
y = 100
w = 50
h = 100
roi = imagen[y:y+h,x:x+w]

fig,ax = plt.subplots(1,2,figsize=(6,4))
intensity_profile_hor = np.mean(roi, axis=0)  # promedio horizontal
ax[0].plot(intensity_profile_hor)

intensity_profile_ver = np.mean(roi, axis=1)  # promedio vertical
ax[1].plot(intensity_profile_ver)

## Código para cambiar de modelo

In [ ]:
B, G, R = cv.split(imagen)
imagen = cv.merge([R,G,B])

image_RGB = cv.cvtColor(imagen, cv.COLOR_RGB2BGR)
image_BGR = cv.cvtColor(imagen, cv.COLOR_BGR2RGB)

image_HSV = cv.cvtColor(imagen, cv.COLOR_BGR2HSV)
image_BGR = cv.cvtColor(imagen, cv.COLOR_HSV2BGR)

In [ ]:
def BGR_a_HSI(image):
    image = image.astype(np.float32) / 255.0
    # Obtener R, G, B
    B, G, R = cv.split(image)

    # Calcular Intensidad como promedio de R, G, y B
    I = (R + G + B) / 3.0

    # Calcular Saturación como 1 - m/I
    min_BGR = np.min([R, G, B], axis=0)
    S = 1 - (3.0 * min_BGR) / (R + G + B + 1e-6)

    # Calculas Tono
    num = 0.5 * ((R - G) + (R - B))
    den = np.sqrt((R - G)**2 + (R - B) * (G - B))
    theta = np.arccos(np.clip(num / (den + 1e-6), -1.0, 1.0))
    H = theta.copy()
    H[B > G] = 2 * np.pi - H[B > G]
    H = H / (2 * np.pi)

    # Combinar los canales H, S, I en una sola imagen
    hsi_image = np.zeros_like(image)
    hsi_image[:, :, 0] = H
    hsi_image[:, :, 1] = S
    hsi_image[:, :, 2] = I

    return hsi_image

In [ ]:
def HSI_a_RGB(hsi_image):
    # Obtener H, S, I
    H, S, I = hsi_image[:, :, 0], hsi_image[:, :, 1], hsi_image[:, :, 2]

    rgb_image = np.zeros_like(hsi_image)
    for x in range(hsi_image.shape[0]):  # Por cada pixel
        for y in range(hsi_image.shape[1]):
            h = H[x, y] * 2 * np.pi  # Convertir H de [0, 1] a [0, 2*pi]
            s = S[x, y]
            i = I[x, y]

            if h >= 0 and h < 2 * np.pi / 3:
                b = i * (1 - s)
                r = i * (1 + (s * np.cos(h)) / np.cos(np.pi / 3 - h))
                g = 3 * i - (r + b)
            elif h >= 2 * np.pi / 3 and h < 4 * np.pi / 3:
                h = h - 2 * np.pi / 3
                r = i * (1 - s)
                g = i * (1 + (s * np.cos(h)) / np.cos(np.pi / 3 - h))
                b = 3 * i - (r + g)
            else:
                h = h - 4 * np.pi / 3
                g = i * (1 - s)
                b = i * (1 + (s * np.cos(h)) / np.cos(np.pi / 3 - h))
                r = 3 * i - (g + b)

            # Mergeo los canales calculados, clipeo a [0,1] para la salida
            rgb_image[x, y, 0] = np.clip(r, 0, 1)
            rgb_image[x, y, 1] = np.clip(g, 0, 1)
            rgb_image[x, y, 2] = np.clip(b, 0, 1)

    return rgb_image

In [ ]:
def complemento_color(img_original):
    # Convertir la imagen a HSV
    img_hsv = cv.cvtColor(img_original, cv.COLOR_BGR2HSV)

    # Copiar la imagen original
    img_complemento = img_hsv.copy()

    # Calcular el complemento del color para cada píxel
    H, S, V = cv.split(img_complemento)

    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            H[i,j] = (H[i,j] + 90)%180

    # Convertir la imagen de vuelta a RGB para visualización
    img_complemento = cv.merge([H,S,V])
    img_complemento_RGB = cv.cvtColor(img_complemento, cv.COLOR_HSV2RGB)

    return img_complemento_RGB

## Transformaciones

In [ ]:
def LUT(a, c, r=[0,256]):         # por default de 0 a 255, si le paso parámetro hace en un tramo específico
    r = np.arange(r[0],r[1],1)    # arma vector con paso 1 para aplicar TL punto a punto
    s = np.multiply(r,a) + c      # TL
    s = np.where(s>255,255,s)   # mayor que 255: 255
    s = np.where(s<0,0,s)       # menor que 0: 0
    return s

def LUT_tramos(tramos):
    s = np.arange(0,256,1)  # inicializo formato de salida
    for tramo in tramos:    # para cada tramo que definí tengo: [ganancia, offset, ini, fin]
      s[tramo[2]:tramo[3]] = LUT(tramo[0],tramo[1],[tramo[2],tramo[3]])  # uso función general que armé antes
    return s

def t_log():
  r = np.arange(256)
  s = np.log10(1+r)
  return s
# mapeo_log = t_log()
# salida_log = mapeo_log[imagen[:]]

def t_pow(gamma):
    r = np.arange(256)/255.0  # normalizo para aplicar potencia
    s = pow(r,gamma)          # aplico potencia
    return s*255.0            # devuelvo al rango 0:255
#mapeo_pow = t_pow(gamma)
#salida_pow = mapeo_pow[imagen[:]]

## Operaciones

In [ ]:
def suma(imagenes):
  salida = imagenes.sum(axis=0)              # sumo
  salida[:] = salida[:]/imagenes.shape[0]    # normalizo
  return salida

def diferencia(img1,img2):
  salida = img1-img2
  salida[:] = (salida[:] + 255)/2                     # método 1
  # salida[:] = (salida[:] - np.min(salida))/255      # método 2
  return salida

def multiplicacion(img,mascara):
  return np.multiply(img,mascara)

## Histograma

In [ ]:
def histograma(imagen,mascara=None):
    return cv.calcHist([imagen], [0], mascara, [256], [0,256])

def ecualizarHistograma(imagen):
    return cv.equalizeHist(imagen)

def claheHist(imagen):
    clahe = cv.createCLAHE(clipLimit=20.0, tileGridSize=(16,16))
    return clahe.apply(imagen)

def correlacionHisto(imagen1,imagen2):
    hist1 = cv.calcHist([imagen1], [0], None, [256], [0, 256])
    hist2 = cv.calcHist([imagen2], [0], None, [256], [0, 256])
    return cv.compareHist(hist1, hist2, cv.HISTCMP_CORREL)

def ecualizacion_RGB(imagen): # recibe imagen en bgr
  imagen_RGB = cv.cvtColor(imagen, cv.COLOR_BGR2RGB)
  r, g, b = cv.split(imagen_RGB)             # obtengo canales por separado
  r_eq = cv.equalizeHist(r)                  # histogramas de cada canal
  g_eq = cv.equalizeHist(g)
  b_eq = cv.equalizeHist(b)
  imagen_eq = cv.merge([r_eq, g_eq, b_eq])   # mergeo histogramas en la imagen
  return imagen_eq  # retorna imagen en rgb

def ecualizacion_HSV(imagen): # recibe imagen en bgr
  imagen_HSV = cv.cvtColor(imagen, cv.COLOR_BGR2HSV)
  h, s, v = cv.split(imagen_HSV)            # obtengo canales por separado
  v_eq = cv.equalizeHist(v)                 # ecualizo histograma en V
  imagen_eq = cv.merge([h,s,v_eq])          # mergeo los valores orignales de h y s con el histograma de v ecualizado.
  imagen_eq_rgb = cv.cvtColor(imagen_eq, cv.COLOR_HSV2RGB)   # retorna imagen en rgb
  return imagen_eq_rgb

def ecualizacion_HSI(imagen): # recibe imagen en bgr
  imagen_HSI = BGR_a_HSI(imagen)
  h, s, i = cv.split(imagen_HSI)        # obtengo canales por separado
  i_uint8 = (i * 255).astype(np.uint8)
  i_eq = cv.equalizeHist(i_uint8)       # ecualizo I
  i_eq = i_eq.astype(np.float32) / 255.0
  imagen_eq = cv.merge([h,s,i_eq])      # mergeo los valores orignales de h y s con el histograma de i ecualizado.
  imagen_eq_rgb = HSI_a_RGB(imagen_eq)
  return imagen_eq_rgb  # retorna imagen en rgb

## Propiedades

In [ ]:
def calcular_propiedades(histograma): # calculo todas las propiedades para un histograma
    histograma_norm = histograma.ravel()/histograma.sum()  # aplano el vector y normalizo

    media = np.sum(histograma_norm * np.arange(256))
    varianza = np.sum(histograma_norm * ((np.arange(256) - media) ** 2))
    asimetria = np.sum(histograma_norm * ((np.arange(256) - media) ** 3)) / (varianza ** (3/2))
    energia = np.sum(histograma_norm ** 2)
    entropia = -np.sum(histograma_norm * np.log2(histograma_norm + 1e-7)) # agrego 1e-7 para que no divida por 0
    return media, varianza, asimetria, energia, entropia

## Filtros

In [ ]:
def filtroMascara(imagen,mask):
    return cv.filter2D(imagen,-1,mask)

# filtro pasa-bajo:
def filtroBox(imagen,kernel_size):
    return cv.boxFilter(imagen,-1,kernel_size,cv.BORDER_ISOLATED )

def filtroGausiano(imagen,kernel_size,sigma_x,sigma_y):
    return cv.GaussianBlur(imagen, kernel_size, sigma_x, sigma_y)

def filtroMedia(imagen, kernel_size):
    return cv.medianBlur(imagen, kernel_size)

# KERNEL POSTIIVO QUE SUMA 1
# PESOS TODOS IGUALES (promediadores -> img borrosa)
mask1 = np.array([[1/9, 1/9, 1/9],
                  [1/9, 1/9, 1/9],   # 1 + 1 + 1 + 1 - 3 = 1
                  [1/9, 1/9, 1/9]])
# PESOS DESIGUALES
mask2 = np.array([[0, 1/8, 0],
                  [1/8, 1/2, 1/8],   # 1 + 1 + 1 + 1 - 3 = 1
                  [0, 1/8, 0]])

# filtro pasa-alto:
# KERNEL POS Y NEG QUE SUMA 1 (no altera bajas frec)
mask1 = np.array([[0, 1, 0],
                  [1, -3, 1],   # 1 + 1 + 1 + 1 - 3 = 1
                  [0, 1, 0]])
mask2 = np.array([[-1, -2, -1],
                  [0, 1, 0],    # 1 - 1 - 2 - 1 + 1 + 2 + 1 = 1
                  [1, 2, 1]])
# KERNEL POS Y NEG QUE SUMA 0 (elimina bajas frec)
mask3 = np.array([[1, 1, 1],
                  [1, -8, 1],
                  [1, 1, 1]])
mask4 = np.array([[-1, -2, -1],
                  [0, 0, 0],
                  [1, 2, 1]])

## Segmentaciones

In [ ]:
def segmentacion_rgb(img,roi,R):
    # obtenemos información de la roi para generar máscara
    BGR_roi = cv.split(roi)
    maxB = np.argmax(np.histogram(BGR_roi[0],np.arange(256))[0])  # obtenemos valor más frecuente de rojo
    maxG = np.argmax(np.histogram(BGR_roi[1],np.arange(256))[0])  # valor más frecuente de verde
    maxR = np.argmax(np.histogram(BGR_roi[2],np.arange(256))[0])  # valor más frecuente de azul

    # generar máscara:
    H,W,_ = img.shape
    maskBGR = np.zeros((H,W),np.uint8)

    for i in range(H):  # recorremos píxel a píxel
        for j in range(W):
            if ((maxB - img[i][j][0])**2 + (maxG - img[i][j][1])**2 + (maxR - img[i][j][2])**2) <= R**2: # si los valores de rojo, verde y azul del píxel i,j
                                                                                                         # tienen una distancia euclidea menor a R con los
                                                                                                         # valores más frecuentes de la ROI, me interesa.
                maskBGR[i][j]=255 # le pongo valor 255 a la máscara en ese píxel, si no entra acá ese píxel queda en 0 y va a descartarlo cunado aplique la máscara.
    return maskBGR

#maskBGR = segmentacion_rgb(futbol,roi,R=93)
#futbol_segm_bgr = cv.bitwise_and(futbol,futbol,mask=maskBGR)

def segmentacion_hsv(img,roi,deltaH,deltaS):
    # obtenemos los canales de la imagen y de la ROI.
    HSV = cv.split(cv.cvtColor(roi, cv.COLOR_BGR2HSV))
    HSV_img = cv.split(cv.cvtColor(img, cv.COLOR_BGR2HSV))

    # buscamos pico del histograma (valor más frecuente) para los dos canales
    maxH = np.argmax(np.histogram(HSV[0],np.arange(256))[0])
    maxS = np.argmax(np.histogram(HSV[1],np.arange(256))[0])

    # generar máscara:
    H,W,_ = img.shape
    maskHSV = np.zeros((H,W),np.uint8)

    for i in np.arange(H):  # recorremos píxel a píxel
        for j in np.arange(W):
            # hacemos 1 en la máscara sólo a los píxeles cuyos valores de H y S están dentro del rectángulo definido por los límites:
            # (maxH - deltaH, maxH + deltaH) en una dimensión correspondiente al hue o tono
            # (maxS - deltaS, maxS + deltaS) en la otra dimensión correspondiente a la saturación
            if (HSV_img[0][i,j] >= maxH-deltaH) and (HSV_img[0][i,j] <= maxH+deltaH) and (HSV_img[1][i,j] >=maxS-deltaS) and (HSV_img[1][i,j] <=maxS+deltaS):
                maskHSV[i,j] = 255
    return maskHSV

#maskHSV = segmentacion_hsv(futbol,roi,deltaH=18,deltaS=105)
#futbol_segm_hsv = cv.bitwise_and(futbol,futbol,mask=maskHSV)

## Fourier

In [ ]:
def get_dft(image):
    dft = cv.dft(np.float32(image), flags=cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude_spectrum = 20 * np.log(cv.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]) + 1e-6) # sumo para q no haga log(0)
    return magnitude_spectrum

def get_fft(image):
    img_fft = np.fft.fft2(image)
    img_fft_shift = np.fft.fftshift(img_fft)
    magnitude_spectrum = np.abs(img_fft_shift)
    # logaritmo para mayor contraste:
    log_magnitude_spectrum = np.log(magnitude_spectrum + 1e-6) # sumo epsilon p/ evitar log(0)
    return log_magnitude_spectrum

def get_mag_phase(imagen):
  f = np.fft.fft2(imagen)
  fshift = np.fft.fftshift(f)
  magnitud = np.abs(fshift)
  fase = np.angle(fshift)
  return magnitud, fase

def ifft(espectro):
    imagen=np.fft.fftshift(espectro)
    imagen=np.abs(np.fft.ifft2(imagen))
    # imagen=np.fft.fftshift(imagen)
    imagen=np.real(imagen)
    return imagen

## Filtros frecuenciales

In [ ]:
def apply_filter(image, filter_mask):
    dft_shift = get_dft(image)
    filtered_dft = dft_shift * filter_mask[:, :, np.newaxis]  # Aplicar filtro
    f_ishift = np.fft.ifftshift(filtered_dft)
    img_back = cv.idft(f_ishift)
    img_back = cv.magnitude(img_back[:, :, 0], img_back[:, :, 1])
    img_back = np.clip(img_back / np.max(img_back), 0, 1)  # Normalizar para visualización
    return img_back

def ideal_low_pass_filter(shape, cutoff_frequency):
    rows, cols = shape                                        # Consigue las dimensiones de la imagen
    center_row, center_col = rows // 2, cols // 2             # Consigue el centro de la imagen
    filter_mask = np.zeros((rows, cols), dtype=np.float32)    # Crea una matriz de igual tamaño que la imagen pero llena de 0s
    cv.circle(filter_mask, (center_col, center_row),
               cutoff_frequency, 1, thickness=-1)             # Dibuja un circulo de readio "frecuencia de corte" en el centro de la imagen
    return filter_mask                                        # Retorna la máscara

def ideal_high_pass_filter(shape, cutoff_frequency):
    rows, cols = shape                                        # Consigue las dimensiones de la imagen
    center_row, center_col = rows // 2, cols // 2             # Consigue el centro de la imagen
    filter_mask = np.ones((rows, cols), dtype=np.float32)    # Crea una matriz de igual tamaño que la imagen pero llena de 0s
    cv.circle(filter_mask, (center_col, center_row),
               cutoff_frequency, 0, thickness=-1)             # Dibuja un circulo de readio "frecuencia de corte" en el centro de la imagen
    return filter_mask                                        # Retorna la máscara

def butterworth_low_pass_filter(shape, cutoff_frequency, order):
    rows, cols = shape
    center_row, center_col = rows // 2, cols // 2
    filter_mask = np.zeros((rows, cols), dtype=np.float32)

    for u in range(rows):
        for v in range(cols):
            distance = np.sqrt((u - center_row) ** 2 + (v - center_col) ** 2)           # Calcula la distancia al centro de la imagen D(u,v)
            filter_mask[u, v] = 1 / (1 + (distance / cutoff_frequency) ** (2 * order))  # Crea la máscara para Butterwort

    return filter_mask

def butterworth_high_pass_filter(shape, cutoff_frequency, order):
    rows, cols = shape
    center_row, center_col = rows // 2, cols // 2
    filter_mask = np.zeros((rows, cols), dtype=np.float32)

    for u in range(rows):
        for v in range(cols):
            distance = np.sqrt((u - center_row) ** 2 + (v - center_col) ** 2)                 # Calcula la distancia al centro de la imagen D(u,v)
            filter_mask[u, v] = 1 - (1 / (1 + (distance / cutoff_frequency) ** (2 * order)))  # Crea la máscara para Butterwort

    return filter_mask

def gaussian_low_pass_filter(shape, sigma):
    rows, cols = shape
    center_row, center_col = rows // 2, cols // 2
    filter_mask = np.zeros((rows, cols), dtype=np.float32)

    for u in range(rows):
        for v in range(cols):
            distance = np.sqrt((u - center_row) ** 2 + (v - center_col) ** 2)
            filter_mask[u, v] = np.exp(-(distance**2 / (2 * sigma**2)))

    return filter_mask

def gaussian_high_pass_filter(shape, sigma):
    rows, cols = shape
    center_row, center_col = rows // 2, cols // 2
    filter_mask = np.zeros((rows, cols), dtype=np.float32)

    for u in range(rows):
        for v in range(cols):
            distance = np.sqrt((u - center_row) ** 2 + (v - center_col) ** 2)
            filter_mask[u, v] = 1 - np.exp(-(distance**2 / (2 * sigma**2)))

    return filter_mask

# Filtro homomórfico: claros más oscuros y oscuros más claros
def homomorphic_filter(img, rl=0.05, rh=0.5, c=25, orden=2):
    rows, cols = img.shape

    # USAMOS FILTRO GAUSSIANO:
    x = np.linspace(-1, 1, cols)
    y = np.linspace(-1, 1, rows)
    X, Y = np.meshgrid(x, y)
    d = np.sqrt(X**2 + Y**2)
    mask = (rh - rl) * (1 - np.exp(-c * d**orden)) + rl

    # APLICAR FILTRO:
    img_log = np.log1p(img)
    img_fft = np.fft.fft2(img_log)
    img_fft_shift = np.fft.fftshift(img_fft)
    img_fft_filt = img_fft_shift * mask
    img_filt = np.real(np.fft.ifft2(np.fft.ifftshift(img_fft_filt)))
    img_exp = np.expm1(img_filt)
    img_norm = cv.normalize(img_exp, None, 0, 255, cv.NORM_MINMAX)

    img_filtered = np.uint8(img_norm)

    return img_filtered

def AltaPotencia(pasaAltos, A):
  altaPotencia = (A-1)+pasaAltos
  return altaPotencia

def EnfasisAltaFrecuencia(a,b,mascaraPA):
  MascaraEAF = a+b*mascaraPA
  return MascaraEAF

def AltaFrecuenciaimg(img,A,imgPA):
  img_AF = (A-1)*img+imgPA
  return img_AF

## Rotación

In [ ]:
angulo = 20
imagen_rotada = imutils.rotate(imagen, angulo)

In [ ]:
def rotate_to_angle(original_image, image_to_rotate):

    # Aplicar la transformada de Fourier a las imágenes
    fft_original = np.real(np.fft.fft2(original_image)).astype(np.uint8)
    fft_to_rotate = np.real(np.fft.fft2(image_to_rotate)).astype(np.uint8)

    # Calcular el espectro de potencia
    power_spectrum_original = np.abs(fft_original) ** 2
    power_spectrum_to_rotate = np.abs(fft_to_rotate) ** 2

    # Calcular la correlación cruzada entre los espectros de potencia
    cross_correlation = cv.matchTemplate(power_spectrum_original, power_spectrum_to_rotate, cv.TM_CCORR_NORMED)

    # Encontrar la posición del valor máximo en la matriz de correlación cruzada
    _, max_val, _, max_loc = cv.minMaxLoc(cross_correlation)

    # Calcular el desplazamiento en píxeles desde la posición central hasta la posición del valor máximo
    rows, cols = cross_correlation.shape
    displacement = np.array(max_loc) - np.array((cols // 2, rows // 2))

    # Calcular el ángulo de rotación correspondiente al desplazamiento
    angle = np.degrees(np.arctan2(displacement[1], displacement[0]))

    return (180 + angle)  # como el ángulo obtenido en el dominio de la frecuencia está entre [-180,180], sumo 180 para tener [0,360]

## Ruido

In [ ]:
def ruido_gaussiano(image, mean=100, stdev=10):
    gauss = np.random.normal(mean, stdev, image.shape)
    noise_image = image + gauss
    return np.uint8(noise_image)

def ruido_uniforme(image, low=-5, high=5):
    uniform = np.random.uniform(low, high, image.shape)
    noise_image = image + uniform
    return np.uint8(noise_image)

def ruido_impulsivo(image, salt_prob=0.01, pepper_prob=0.01):
    noise_image = image.copy()
    probs = np.random.random(noise_image.shape[:2])
    noise_image[probs < salt_prob] = 255
    noise_image[probs > 1 - pepper_prob] = 0
    return np.uint8(noise_image)

def ruido_exponencial(image, scale=25):
    exponencial = np.random.exponential(scale, image.shape)
    noise_image = image + exponencial
    return np.uint8(noise_image)

## Filtros de medias

In [ ]:
def geometric_mean_filter(image, filter_size):
    im_H, im_W = image.shape
    padsize = (filter_size - 1) // 2 # Calcular el tamaño del padding necesario

    # Añadir padding a la imagen para manejar los bordes
    # cv.copyMakeBorder se usa para añadir bordes constantes de valor 1 alrededor de la imagen
    im_p = cv.copyMakeBorder(np.copy(image), padsize, padsize, padsize, padsize, cv.BORDER_CONSTANT, value=1).astype(np.float64)

    im_f_mg = np.zeros(image.shape)

    # Aplicar el filtro de media geométrica
    for i in range(im_H):
        for j in range(im_W):
            window = im_p[i:i + filter_size, j:j + filter_size] # Extraer la ventana de la imagen con padding
            product = np.prod(window)
            im_f_mg[i, j] = product ** (1 / (filter_size ** 2))

    return np.uint8(im_f_mg) # Convertir la imagen filtrada al tipo uint8 para asegurar que los valores sean válidos en el rango [0, 255]


def contrarmonic_filter(image, ksize, Q): # función recibe imagen, tamaño de la ventana del filtro (debe ser impar) y Q es orden del filtro contra-armónico
    im_H, im_W = image.shape
    padsize = (ksize - 1) // 2 # Calcular el tamaño del padding necesario

    # Añadir padding a la imagen para manejar los bordes
    # cv.copyMakeBorder se usa para añadir bordes de ceros alrededor de la imagen
    im_p = cv.copyMakeBorder(np.copy(image), padsize, padsize, padsize, padsize, cv.BORDER_CONSTANT, value=0).astype(np.float64)

    im_f_mch = np.zeros(image.shape)

    # Aplicar el filtro contrarmónico
    for i in range(im_H):
        for j in range(im_W):
            window = im_p[i:i + ksize, j:j + ksize] # Extraer la ventana de la imagen con padding
            numerator = np.sum(window ** (Q + 1))
            denominator = np.sum(window ** Q)

            # Calcular el valor filtrado, manejando divisiones por cero
            im_f_mch[i, j] = numerator / (denominator + 1e-6)

    return np.uint8(im_f_mch) # Convertir la imagen filtrada al tipo uint8 para valores válidos en el rango [0, 255]

## Filtros de orden

In [ ]:
def filtro_mediana(img,kernel_size = 3):
  output = cv.medianBlur(img,kernel_size)
  return np.uint8(output)

def filtro_punto_medio(image, kernel_size = 3):
  output = np.zeros(image.shape)
  H, W = image.shape
  padsize = (kernel_size-1)//2
  im_p = cv.copyMakeBorder(np.copy(image), *[padsize]*4, cv.BORDER_CONSTANT,0).astype(np.float64)

  for i in range(H):
      for j in range(W):
          vMin, vMax = cv.minMaxLoc(im_p[i:i+kernel_size,j:j+kernel_size])[0:2]
          output[i, j] = (vMin+vMax)/2
  return np.uint8(output)

def filtro_media_alfa(img, kernel_size = 3, d = 8):
  output = np.zeros(img.shape)
  im_H, im_W = img.shape
  padsize = (kernel_size-1)//2
  im_p = cv.copyMakeBorder(np.copy(img), *[padsize]*4, cv.BORDER_CONSTANT,0).astype(np.float64)

  for i in range(im_H):
      for j in range(im_W):
          S = np.sort(im_p[i:i+kernel_size,j:j+kernel_size].flatten())
          S = S[d//2:S.shape[0]-d//2]
          output[i, j] = np.sum(S)/S.shape[0]

  return np.uint8(output)

## Mean-Squared Error

In [ ]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

## Detección de bordes:

In [ ]:
def prewitt_edge_detection(image):
    prewitt_x = cv.filter2D(image, -1, np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]))
    prewitt_y = cv.filter2D(image, -1, np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]))
    prewitt_edges = cv.bitwise_or(prewitt_x, prewitt_y)
    _, binary_edges = cv.threshold(prewitt_edges, 50, 255, cv.THRESH_BINARY)
    return binary_edges

In [ ]:
# Función para normalizar y actualizar la imagen con los nuevos parámetros
def update_sobel(ddepth, dx, ksize):
    global image
    edges = cv.Sobel(image, ddepth, dx, 1-dx, ksize=ksize)
    plt.imshow(edges, cmap='gray', vmin=0, vmax=255)
    plt.title('Sobel Edges')
    plt.axis('off')
    plt.show()

# Crear controles interactivos
ddepth_widget = RadioButtons(options=[('CV_8U', cv.CV_8U), ('CV_64F', cv.CV_64F)], description='ddepth:')
dx_widget = RadioButtons(options=[('dx', 1), ('dy', 0)], description='Derivative:')
ksize_widget = IntSlider(min=-1, max=31, step=2, value=3, description='ksize:')

# Llamar a la función interact para generar la interfaz interactiva
interact(update_sobel, ddepth=ddepth_widget, dx=dx_widget, ksize=ksize_widget);

In [ ]:
# Función para actualizar la imagen con los nuevos parámetros
def update_laplacian(ksize):
    global image
    edges_laplacian = cv.Laplacian(image, cv.CV_64F, ksize=ksize)
    plt.imshow(edges_laplacian, cmap='gray')
    plt.title('Bordes Laplacianos')
    plt.axis('off')
    plt.show()

# Crear control deslizante interactivo
ksize_slider = IntSlider(min=1, max=31, step=2, value=3, description='Tamaño del kernel')

# Llamar a la función interact para generar la interfaz interactiva
interact(update_laplacian, ksize=ksize_slider);

In [ ]:
# Función para actualizar la imagen con los nuevos parámetros
def update_canny(threshold1, threshold2, l2gradient):
    global image
    # imagen_suavizada = cv.GaussianBlur(imagen_gris, (5, 5), 0)  # filtro gaussiano para mejor umbralizado (varía según img)
    edges = cv.Canny(image, threshold1, threshold2, L2gradient=bool(l2gradient))
    plt.imshow(edges, cmap='gray')
    plt.title('Bordes Canny')
    plt.axis('off')
    plt.show()

# Crear controles interactivos
threshold1_slider = IntSlider(min=0, max=255, value=50, description='Threshold 1:')
threshold2_slider = IntSlider(min=0, max=255, value=150, description='Threshold 2:')
l2gradient_checkbox = Checkbox(value=False, description='L2gradient')

# Llamar a la función interact para generar la interfaz interactiva
interact(update_canny, threshold1=threshold1_slider, threshold2=threshold2_slider, l2gradient=l2gradient_checkbox);

In [ ]:
# cv.HoughLinesP(	image, rho, theta, threshold[, lines[, minLineLength[, maxLineGap]]]	)
image2 = cv.imread('/content/drive/My Drive/PDI/letras1.tif', cv.IMREAD_GRAYSCALE)
# imagen_suavizada = cv.GaussianBlur(imagen_gris, (5, 5), 0)  # filtro gaussiano para mejor umbralizado (varía según img)
image2_canny = cv.Canny(image2, 50, 200, None)  #  en documentación: "Input image should be a binary image, so apply threshold or use canny edge detection before applying hough transform"

def updateHough(set_threshold, set_mintheta, set_maxtheta):
  imageHough = cv.cvtColor(image2_canny, cv.COLOR_GRAY2BGR)
  lines = cv.HoughLines(image2_canny, rho = 1, theta = np.pi/180, threshold = set_threshold, min_theta = set_mintheta, max_theta = set_maxtheta)

  if lines is not None:
      for line in lines:
          rho, theta = line[0]
          cos_theta, sin_theta = np.cos(theta), np.sin(theta)
          x0, y0 = rho * cos_theta, rho * sin_theta
          x1, y1 = int(x0 + 1000 * (-sin_theta)), int(y0 + 1000 * cos_theta)
          x2, y2 = int(x0 - 1000 * (-sin_theta)), int(y0 - 1000 * cos_theta)
          cv.line(imageHough, (x1, y1), (x2, y2), (255, 0, 172), 3, cv.LINE_AA)

  plt.imshow(imageHough)
  plt.title(f'threshold = {set_threshold}, min_theta = {set_mintheta}, max_theta = {set_maxtheta}')
  plt.axis('off')
  plt.show()

slider_threshold = IntSlider(min=0, max=200, value=53, description='Threshold:')
slider_mintheta = FloatSlider(min=0.0, max=3.14, step=0.01, value=0.0, description='Min Theta:')
slider_maxtheta = FloatSlider(min=0.0, max=3.14, step=0.01, value=2.66, description='Max Theta:')

# para que los sliders no tiren error (min tiene que ser menor a max sino rompe el HoughLines)
def check_theta(change):
    if slider_mintheta.value > slider_maxtheta.value:
        slider_maxtheta.value = slider_mintheta.value
slider_mintheta.observe(check_theta, 'value')
slider_maxtheta.observe(check_theta, 'value')

interact(updateHough, set_threshold = slider_threshold, set_mintheta = slider_mintheta, set_maxtheta = slider_maxtheta);

In [ ]:
# cv.HoughLinesP(	image, rho, theta, threshold[, lines[, minLineLength[, maxLineGap]]]	)
image2 = cv.imread('/content/drive/My Drive/PDI/letras1.tif', cv.IMREAD_GRAYSCALE)
#imagen_suavizada = cv.GaussianBlur(imagen_gris, (5, 5), 0)  # filtro gaussiano para mejor umbralizado (varía según img)
image2_canny = cv.Canny(image2, 50, 200, None)  #  en documentación: "Input image should be a binary image, so apply threshold or use canny edge detection before applying hough transform"

def updateHoughP(set_threshold, set_minLength, set_maxGap):
  imageHoughP = cv.cvtColor(image2_canny, cv.COLOR_GRAY2BGR)
  lines = cv.HoughLinesP(image2_canny, rho = 1, theta = np.pi/180, threshold = set_threshold, minLineLength = set_minLength, maxLineGap = set_maxGap)

  if lines is not None:
      for line in lines:
          l = line[0]
          cv.line(imageHoughP, (l[0], l[1]), (l[2], l[3]), (255,0,127,0), 3, cv.LINE_AA)
  plt.imshow(imageHoughP)
  plt.title(f'threshold = {set_threshold}, minLineLength = {set_minLength}, maxLineGap = {set_maxGap}')
  plt.axis('off')
  plt.show()

slider_threshold = IntSlider(min=0, max=200, value=35, description='Threshold:')
slider_minLength = IntSlider(min=0, max=200, value=32, description='Min Line Length:')
slider_maxGap = IntSlider(min=0, max=200, value=19, description='Max Line Gap:')

interact(updateHoughP, set_threshold = slider_threshold, set_minLength = slider_minLength, set_maxGap = slider_maxGap);

In [ ]:
img_sitk = sitk.VectorIndexSelectionCast(sitk.ReadImage('/content/drive/My Drive/PDI/bone.tif'), 0)

LOWER = 80                         # umbral mínimo
UPPER = 255                        # umbral máximo
SEEDLIST = [(230,150),(150,45)]    # semillas (podría ser con click pero...)

seg = sitk.ConnectedThreshold(img_sitk, seedList=SEEDLIST, lower=LOWER, upper=UPPER)

fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(9,9))
ax[0].imshow(sitk.GetArrayFromImage(img_sitk),cmap="gray")
for seed in SEEDLIST: # grafico las semillas para ver donde las tomé
  ax[0].scatter(seed[0],seed[1])
ax[1].imshow(sitk.GetArrayFromImage(seg), cmap='gray', alpha=0.5)
plt.show()

In [ ]:
latas = cv.imread('/content/drive/My Drive/PDI/latas.png')
latas1 = latas.copy() # salida sin preprocesamiento

latas_gray = cv.cvtColor(latas,cv.COLOR_BGR2GRAY)
latas_blur = cv.medianBlur(latas_gray,5)
circles = cv.HoughCircles(latas_blur, cv.HOUGH_GRADIENT, 1, 250, minRadius=115)

if circles is not None:
    circles = np.uint16(circles)
    for i in circles[0, :]:
        cv.circle(latas1, (i[0],i[1]), i[2], (255, 0, 170), 3)

## Morfología

In [ ]:
EE = np.ones((3,3)).astype(np.uint8)    # cuadrado

EE = np.zeros((5,5)).astype(np.uint8)   # línea diagonal
for i in range(EE.shape[0]):  # pruebo con distintos tamaños de kernel (n,n)
  EE[i,EE.shape[0]-1-i] = 255 # hago la diagonal

ee_cross = cv.getStructuringElement(cv.MORPH_CROSS,(3,3))
ee_rect = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
ee_ellipse = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))

In [ ]:
estrellas = cv.imread("drive/MyDrive/PDI/estrellas.jpg")
estrellas_gris = cv.cvtColor(estrellas,cv.COLOR_BGR2GRAY).astype(np.uint8)
_,estrellas_binario = cv.threshold(estrellas_gris,130,255,cv.THRESH_BINARY)

dst = cv.erode(imagen,EE)
dst = cv.dilate(imagen,EE)
# Apertura: dilate -> erode
# Cierre: erode -> dilate

apertura = cv.morphologyEx(tarjeta_binario,cv.MORPH_OPEN,EE, iteration=2)
cierre = cv.morphologyEx(tarjeta_binario,cv.MORPH_CLOSE,EE, iteration=2)
erosion = cv.morphologyEx(tarjeta_binario,cv.MORPH_ERODE,EE, iteration=2)
dilatacion = cv.morphologyEx(tarjeta_binario,cv.MORPH_DILATE,EE, iteration=2)